In [7]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# Training data is 11 points in [0,1] inclusive regularly spaced
train_x = torch.linspace(0, 1, 100)
# True function is sin(2*pi*x) with Gaussian noise
train_y = torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * 0.2


In [9]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)

In [10]:
# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

training_iter = 50
for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))
    optimizer.step()


Iter 1/50 - Loss: 0.966   lengthscale: 0.693   noise: 0.693
Iter 2/50 - Loss: 0.936   lengthscale: 0.644   noise: 0.644
Iter 3/50 - Loss: 0.903   lengthscale: 0.598   noise: 0.598
Iter 4/50 - Loss: 0.865   lengthscale: 0.555   noise: 0.554
Iter 5/50 - Loss: 0.821   lengthscale: 0.514   noise: 0.513
Iter 6/50 - Loss: 0.771   lengthscale: 0.476   noise: 0.474
Iter 7/50 - Loss: 0.716   lengthscale: 0.440   noise: 0.437
Iter 8/50 - Loss: 0.660   lengthscale: 0.405   noise: 0.402
Iter 9/50 - Loss: 0.606   lengthscale: 0.373   noise: 0.369
Iter 10/50 - Loss: 0.557   lengthscale: 0.343   noise: 0.339
Iter 11/50 - Loss: 0.513   lengthscale: 0.315   noise: 0.310
Iter 12/50 - Loss: 0.473   lengthscale: 0.291   noise: 0.283
Iter 13/50 - Loss: 0.437   lengthscale: 0.271   noise: 0.259
Iter 14/50 - Loss: 0.402   lengthscale: 0.254   noise: 0.236
Iter 15/50 - Loss: 0.369   lengthscale: 0.240   noise: 0.215
Iter 16/50 - Loss: 0.337   lengthscale: 0.228   noise: 0.195
Iter 17/50 - Loss: 0.305   length

In [13]:
test_x = torch.linspace(0, 1, 51).cuda()
model.eval().cuda()
likelihood.eval()

# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    observed_pred = likelihood(model(test_x))
    mean = observed_pred.mean
    lower, upper = observed_pred.confidence_region()

tensor([ 0.0199,  0.1269,  0.2371,  0.3485,  0.4591,  0.5669,  0.6698,  0.7657,
         0.8524,  0.9281,  0.9910,  1.0394,  1.0720,  1.0878,  1.0860,  1.0662,
         1.0284,  0.9729,  0.9006,  0.8125,  0.7100,  0.5951,  0.4697,  0.3363,
         0.1973,  0.0554, -0.0868, -0.2263, -0.3608, -0.4875, -0.6042, -0.7087,
        -0.7993, -0.8745, -0.9331, -0.9743, -0.9977, -1.0033, -0.9915, -0.9628,
        -0.9182, -0.8591, -0.7868, -0.7032, -0.6100, -0.5092, -0.4028, -0.2929,
        -0.1814, -0.0704,  0.0383], device='cuda:0')